In [ ]:
import requests
from lxml import html

import redis
from random import randint
import time

from ipywidgets import Image
import plotly.graph_objs as go

In [ ]:
from mlrepricer import helper, redisdb, helper
from mlrepricer.oldsql import schemas
from mlrepricer.oldsql.database import SQLite
import pandas as pd
import numpy as np

How you get started with redis.

In [ ]:
redisdb.server_start()
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)

In [ ]:
type(r)

Iterate over pictures:

In [ ]:
pictures = dict()
for asin in r.scan_iter():
    if asin.startswith('pic'):
        pictures.update({asin: list(r.smembers(asin))[0]})

In [ ]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

In [ ]:
filter1 = helper.cleanup(df)
norm = helper.normalize(filter1)
norm.head()

In [ ]:
fig = go.FigureWidget(
    data=[
        dict(
            x=norm['time_changed'],
            y=norm['price'],
        )
    ],
)

In [ ]:
scatter = fig.data[0]
scatter.text = norm['sellerid']
scatter.hoverinfo = 'text'

In [ ]:
from ipywidgets import HTML
details = HTML()

In [ ]:
def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    details.value = norm.iloc[ind].to_frame().to_html()

scatter.on_hover(hover_fn)

In [ ]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=pictures['pic_B075NJNLJT'],
    layout=Layout(height='400px', width='400px')
)
image_widget

In [ ]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = norm.iloc[ind].to_frame().to_html()
    
    # Update image widget
    model_year = 'pic_' + norm['asin'][ind]
    image_widget.value = m[model_year]

scatter.on_hover(hover_fn)

In [ ]:
from ipywidgets import HBox, VBox
VBox([fig,
      HBox([image_widget, details])])

In [ ]:
# that would be one picture
pic = list(r.smembers('pic_B01F2RLCJ8'))[0]
#r.delete('pic_B01J670I36')
# lets use the list of pictures, you can change the index to play around
Image(value=m[49])

In [ ]:
def download_image(asin):
    country = 'de'
    site = requests.get(f'https://amazon.{country}/dp/{asin}')
    parser = html.fromstring(site.text)
    IMG = "//img[@id='landingImage']"
    image_link = parser.xpath(IMG)[0].get('src')
    image = requests.get(image_link).content
    assert isinstance(image, bytes)
    return image
    
def dump_into_redis(image):
    r.sadd(f'pic_{asin}', image)

In [ ]:
for asin in r.scan_iter():
    if asin != 'updated_asins' and not asin.startswith('pic'):
        if not r.exists('pic_'+asin):
            print(asin)
            time.sleep(randint(2,4))
            try:
                dump_into_redis(download_image(asin))
            except:
                time.sleep(randint(5, 6))